## Вопросно-ответные текста и понимание текста

QA и Reading comprehension - одни из самых актуальных сейчас задач в Nlp. Они 1) очень сложные и 2) легко интерпретируемые, что делает их хорошими тестами на AI. В принципе это разные задачи, но они настолько похожи (в обоих случаях это ответы на вопросы по тексту или вообще), что различия часто не делают. 

За последнее время вышло много крутых и сложных вопросно-ответных датасетов (тут можно найти основные -http://nlpprogress.com/english/question_answering.html)

Другие задачи тоже уже пробуют переделывать под формат вопросно-ответных систем - https://decanlp.com/

Самый популярный датасет - SQUAD от Стэнфорда (https://rajpurkar.github.io/SQuAD-explorer/). На нем тестируют все самые новые нейронки (BERT например). 

Давайте попробуем обучить какую-нибудь нейронку на этих данных.

In [1]:
import json
import numpy as np
import tensorflow as tf

In [2]:
import re

In [3]:
train = json.load(open('train-v2.0.json'))

In [4]:
dev = json.load(open('dev-v2.0.json'))

Не будем заморачиваться с нормализацией.

In [5]:
from string import punctuation
# from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [word for word in words if word]

    return words

Датасет устроен так - есть тексты из википедии, к какому-то параграфу этого текста задан вопрос и из этого же параграфа извлечен ответ. В версии 2.0 добавились также вопросы, на которые нет ответа, что усложняет задачу для модели.

Для каждого ответа даны начало и конец (индексы) в соответствующем параграфе. На этих индексах мы и будем обучаться. 

In [6]:
# corpus = []

In [7]:
contexts = []
questions = []

answers_seqs = []

for instance in train['data']:
    for paragraph in instance['paragraphs']:
        context = paragraph['context']
#         corpus.append(normalize(context))
        
        for qas in paragraph['qas']:
            question = qas['question']
            
            if qas['is_impossible']:
                matches = re.finditer('\w+', context)
                seq = []
                cont = []
                for i, match in enumerate(matches):
                    seq.append(0)
                    cont.append(match.group(0).lower())
                if seq:
                    contexts.append(cont)
                    answers_seqs.append(seq)
                    questions.append(normalize(question))

            
            unique = set()
            for answer in qas['answers']:
                if answer['text'] in unique:
                    continue
                unique.add(answer['text'])
                start = answer['answer_start']
                end = start + len(answer['text'])
                matches = re.finditer('\w+', context)
                seq = []
                cont = []
                for i, match in enumerate(matches):
                    if match.span()[0] >= start and match.span()[1] <= end:
                        seq.append(1)
                    else:
                        seq.append(0)
                    cont.append(match.group(0).lower())
                if seq:
                    contexts.append(cont)
                    answers_seqs.append(seq)
                    questions.append(normalize(question))
                
#                 corpus.append(normalize(question))

In [8]:
len(contexts)

130319

Для отложенной выборки сохраним исходные параграфы и индексы вопросов. Они пригодятся для тестирования.

In [9]:
contexts_dev = []
questions_dev = []
answers_seqs_dev = []


ids = []
imporssible_ids = []
raw_context = []
true_ans = []
for instance in dev['data']:
    for paragraph in instance['paragraphs']:
        context = paragraph['context']
#         corpus.append(normalize(context))
        for qas in paragraph['qas']:
            question = qas['question']
            if qas['is_impossible']:
                matches = re.finditer('\w+', context)
                seq = []
                cont = []
                for i, match in enumerate(matches):
                    seq.append(0)
                    cont.append(match.group(0).lower())
                if seq:
                    contexts_dev.append(cont)
                    answers_seqs_dev.append(seq)
                    questions_dev.append(normalize(question))
                    true_ans.append('NONONO')
                    ids.append(qas['id'])
            
            
            for answer in qas['answers'][:1]:
                start = answer['answer_start']
                end = start + len(answer['text'])
                matches = re.finditer('\w+', context)
                seq = []
                cont = []
                for i, match in enumerate(matches):
                    if match.span()[0] >= start and match.span()[1] <= end:
                        seq.append(1)
                    else:
                        seq.append(0)
                    cont.append(match.group(0).lower())
                if seq:
                    contexts_dev.append(cont)
                    answers_seqs_dev.append(seq)
                    questions_dev.append(normalize(question))
                    true_ans.append(answer['text'])
                
                ids.append(qas['id'])
#                 raw_context.append(context)

In [10]:
len(contexts_dev)

11873

In [11]:
# ft = gensim.models.FastText(corpus, size=200, sg=1)

Теперь построим словарь.

In [12]:
vocab = set()
vocab_q = set()

for context in contexts:
    vocab.update(context)

for question in questions:
    vocab_q.update(question)
    
for context in contexts_dev:
    vocab.update(context)

for question in questions_dev:
    vocab_q.update(question)

id2word = {i+1:word for i, word in enumerate(vocab)}
word2id = {word:i for i, word in id2word.items()}

id2word_q = {i+1:word for i, word in enumerate(vocab_q)}
word2id_q = {word:i for i, word in id2word_q.items()}

In [13]:
len(vocab)

81910

Теперь преодразуем все слова индексы и привидем все к одной длине (максимальной).

In [18]:
## КОНТЕКСТ

In [40]:
contexts_le = [[word2id[word] for word in context] for context in contexts]
max_len = max([len(c) for c in contexts])
# max_len = 200

X_train_context = tf.keras.preprocessing.sequence.pad_sequences(contexts_le, max_len, padding='post')

In [41]:
contexts_le_dev = [[word2id.get(word, 0) for word in context] for context in contexts_dev]
X_dev_context = tf.keras.preprocessing.sequence.pad_sequences(contexts_le_dev, max_len, padding='post')

In [21]:
## ВОПРОС

In [42]:
max_len_q

40

In [43]:
questions_le = [[word2id_q[word] for word in question] for question in questions]
max_len_q = max([len(c) for c in questions])

X_train_question = tf.keras.preprocessing.sequence.pad_sequences(questions_le, max_len_q, padding='post')

In [44]:
questions_le_dev = [[word2id_q.get(word, 0) for word in question] for question in questions_dev]
X_dev_question = tf.keras.preprocessing.sequence.pad_sequences(questions_le_dev, max_len_q, padding='post')

Зададим параметры для нейронки.

In [45]:
vocab_size = len(vocab)+1
vocab_size_q = len(vocab_q)+1
embedding_vector_length = 200

In [46]:
y_train = tf.keras.preprocessing.sequence.pad_sequences(answers_seqs, max_len, padding='post')
y_dev = tf.keras.preprocessing.sequence.pad_sequences(answers_seqs_dev, max_len, padding='post')

In [68]:
y_train = y_train.reshape(-1, max_len, 1)
y_dev = y_dev.reshape(-1, max_len, 1)

Теперь самая сложная часть. У нашей нейронки будет два входа. 

**Первый вход** - для параграфа. Он будет эбмедиться и прогоняться через LSTM. Дальше будут передаваться все состояния.

**Второй вход** - для вопроса. Он будет эбмедиться и прогоняться через LSTM. Дальше будет передаваться только последнее состояние.

**Конкатенация** - к каждому вектору состояния во первом входе присоединяется последнее состояние из второго входа. 

**Выход** - каждое состояние передается в бинарный классификатор.

Так как мы решаем задачу классификации - лосс **categorical_crossentropy** (sparse потому что мы не энкодили в ohe вектора)

In [70]:
from tensorflow.keras.layers import *
from tensorflow.keras import *

# Первый вход
context_input = Input(shape=(max_len, ), name='context_input')
emb_c = Embedding(input_dim=vocab_size, output_dim=100, 
              input_length=max_len, trainable=True)(context_input)

lstm_out_c = LSTM(50,  return_sequences=True,)(emb_c)
drop_1 = Dropout(0.3)(lstm_out_c)

# Второй вход
ques_input = Input(shape=(max_len_q, ), name='ques_input')
emb_q = Embedding(input_dim=vocab_size_q, output_dim=50,
              input_length=max_len_q)(ques_input)
lstm_out_q = LSTM(50,return_sequences=False,)(emb_q)
drop_2 = Dropout(0.3)(lstm_out_q)
repeat_vector_q = RepeatVector(max_len)(drop_2)

# merger model
merge_layer = concatenate([drop_1, repeat_vector_q], axis=2)
LSTM_s = LSTM(50,return_sequences=True)(merge_layer)

# Выход 
out = TimeDistributed(Dense(1, activation='sigmoid', ), name='answer')(LSTM_s)


model = Model(inputs=[context_input, ques_input], outputs=[out])


model.compile(optimizer='adam', loss='binary_crossentropy', 
              metrics=[tf.keras.metrics.Precision()])


In [71]:
# ModelCheckpoint сохраняет лучшие версии моделей
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.weights', # названия файла 
                                                monitor='val_f1', # за какой метрикой следить
                                                verbose=1, # будет печатать что происходит
                                                save_weights_only=True, # если нужно только веса сохранить
                                                save_best_only=True, # сохранять только лучшие
                                                mode='max', # если метрика должна расти, то тут max и min если наоборот
                                                save_freq='epoch' # как часто вызывать
                                               )

Попробуем обучаться.

In [72]:
validation_data=({'context_input': X_dev_context,
                  'ques_input':X_dev_question}, 
                 {'answer': y_dev})


training_data=({'context_input': X_train_context,
                'ques_input':X_train_question}, 
                 {'answer': y_train})

model.fit(training_data[0], training_data[1], batch_size=256,  epochs=100, shuffle=True,
          validation_data=(validation_data[0], validation_data[1]),
         )

Epoch 1/100
510/510 [==============================] - 443s 868ms/step - loss: 0.0649 - precision_5: 0.0051 - val_loss: 0.0436 - val_precision_5: 0.0000e+00
Epoch 2/100
510/510 [==============================] - 452s 887ms/step - loss: 0.0504 - precision_5: 0.0000e+00 - val_loss: 0.0434 - val_precision_5: 0.0000e+00
Epoch 3/100
510/510 [==============================] - 443s 868ms/step - loss: 0.0483 - precision_5: 0.0000e+00 - val_loss: 0.0449 - val_precision_5: 0.0000e+00
Epoch 4/100
510/510 [==============================] - 450s 883ms/step - loss: 0.0467 - precision_5: 0.0000e+00 - val_loss: 0.0442 - val_precision_5: 0.0000e+00
Epoch 5/100
510/510 [==============================] - 442s 866ms/step - loss: 0.0452 - precision_5: 0.0000e+00 - val_loss: 0.0471 - val_precision_5: 0.0000e+00
Epoch 6/100
510/510 [==============================] - 441s 866ms/step - loss: 0.0438 - precision_5: 0.0000e+00 - val_loss: 0.0467 - val_precision_5: 0.0000e+00
Epoch 7/100
510/510 [=================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



510/510 [==============================] - 417s 818ms/step - loss: 0.0147 - precision_5: 0.7973 - val_loss: 0.0870 - val_precision_5: 0.0310
Epoch 37/100
510/510 [==============================] - 431s 845ms/step - loss: 0.0140 - precision_5: 0.8076 - val_loss: 0.0897 - val_precision_5: 0.0318
Epoch 38/100
510/510 [==============================] - 423s 830ms/step - loss: 0.0136 - precision_5: 0.8119 - val_loss: 0.0895 - val_precision_5: 0.0334
Epoch 39/100
510/510 [==============================] - 427s 836ms/step - loss: 0.0134 - precision_5: 0.8136 - val_loss: 0.0890 - val_precision_5: 0.0322
Epoch 40/100
510/510 [==============================] - 579s 1s/step - loss: 0.0138 - precision_5: 0.8035 - val_loss: 0.0904 - val_precision_5: 0.0311
Epoch 41/100
510/510 [==============================] - 559s 1s/step - loss: 0.0126 - precision_5: 0.8251 - val_loss: 0.0936 - val_precision_5: 0.0337
Epoch 42/100
 86/510 [====>.........................] - ETA: 7:01 - loss: 0.0114 - precision_5:

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



510/510 [==============================] - 460s 902ms/step - loss: 0.0119 - precision_5: 0.8328 - val_loss: 0.0945 - val_precision_5: 0.0339
Epoch 44/100
510/510 [==============================] - 439s 861ms/step - loss: 0.0115 - precision_5: 0.8390 - val_loss: 0.0952 - val_precision_5: 0.0330
Epoch 45/100
510/510 [==============================] - 438s 858ms/step - loss: 0.0111 - precision_5: 0.8471 - val_loss: 0.0939 - val_precision_5: 0.0327
Epoch 46/100
510/510 [==============================] - 441s 865ms/step - loss: 0.0108 - precision_5: 0.8492 - val_loss: 0.0953 - val_precision_5: 0.0340
Epoch 47/100
510/510 [==============================] - 449s 881ms/step - loss: 0.0105 - precision_5: 0.8542 - val_loss: 0.0968 - val_precision_5: 0.0346
Epoch 48/100
510/510 [==============================] - 442s 867ms/step - loss: 0.0104 - precision_5: 0.8549 - val_loss: 0.0982 - val_precision_5: 0.0351
Epoch 49/100
510/510 [==============================] - 449s 880ms/step - loss: 0.0101 - 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



510/510 [==============================] - 444s 871ms/step - loss: 0.0087 - precision_5: 0.8790 - val_loss: 0.0996 - val_precision_5: 0.0346
Epoch 56/100
510/510 [==============================] - 445s 872ms/step - loss: 0.0085 - precision_5: 0.8809 - val_loss: 0.0984 - val_precision_5: 0.0342
Epoch 57/100
510/510 [==============================] - 442s 867ms/step - loss: 0.0083 - precision_5: 0.8842 - val_loss: 0.1029 - val_precision_5: 0.0328
Epoch 58/100
510/510 [==============================] - 439s 861ms/step - loss: 0.0080 - precision_5: 0.8875 - val_loss: 0.1033 - val_precision_5: 0.0332
Epoch 59/100
510/510 [==============================] - 454s 890ms/step - loss: 0.0078 - precision_5: 0.8916 - val_loss: 0.1016 - val_precision_5: 0.0358
Epoch 60/100
510/510 [==============================] - 451s 884ms/step - loss: 0.0077 - precision_5: 0.8935 - val_loss: 0.1018 - val_precision_5: 0.0359
Epoch 61/100
510/510 [==============================] - 447s 876ms/step - loss: 0.0074 - 

KeyboardInterrupt: 

Посмотрим, какие ответы предсказываются.

In [73]:
preds = model.predict(validation_data[0], verbose=1)

372/372 [==============================] - 25s 67ms/step


По индексам достаем текст из нетронутых текстов из отложенной выборки.

In [74]:
pred_dict = {}
for i in range(len(preds)):
    cont = contexts_dev[i]
    q = questions_dev[i]
    answer = []
    for j, pred in enumerate(preds[i].flatten()):
        if pred > 0.3 and j < len(cont):
            answer.append(cont[j])
#     span = cont[starts_pred[i]:int(starts_pred[i]+ends_pred[i])]
    if answer:
        pred_dict[ids[i]] = [' '.join(cont), ' '.join(q), 
                             ' '.join(answer), true_ans[i]]

# for idx in imporssible_ids:
#     pred_dict[idx] = ""

In [75]:
pred_dict

{'56ddde6b9a695914005b9628': ['the normans norman nourmands french normands latin normanni were the people who in the 10th and 11th centuries gave their name to normandy a region in france they were descended from norse norman comes from norseman raiders and pirates from denmark iceland and norway who under their leader rollo agreed to swear fealty to king charles iii of west francia through generations of assimilation and mixing with the native frankish and roman gaulish populations their descendants would gradually merge with the carolingian based cultures of west francia the distinct cultural and ethnic identity of the normans emerged initially in the first half of the 10th century and it continued to evolve over the succeeding centuries',
  'in what country is normandy located',
  'the normans',
  'France'],
 '5ad39d53604f3c001a3fe8d3': ['the normans norman nourmands french normands latin normanni were the people who in the 10th and 11th centuries gave their name to normandy a regi